In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

In [9]:
from accounts import Account

In [ ]:
account = Account.get("Al")
account

In [ ]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

In [ ]:
account.report()

In [ ]:
account.list_transactions()

### Now we write an MCP server and use it directly!

In [3]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [ ]:
mcp_tools

In [10]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Al and my account is under the name Al. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [ ]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


### Now let's build our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
request = "My name is Al and my account is under the name Al. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("al")
print(context)

In [ ]:
from accounts import Account
Account.get("al").report()